In [1]:
import requests
from datetime import datetime
import pandas as pd

In [2]:
df=pd.read_csv('df5.csv')

In [3]:
import json
def get_codeforces_rating_history(username):
    url = f"https://codeforces.com/api/user.rating?handle={username}"
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            data = response.json()
            if data["status"] == "OK":
                return data["result"]
            else:
                print(f"API Error: {data.get('comment', 'No specific error message provided')}")
        except json.JSONDecodeError:
            print("Invalid JSON response from the API")
    else:
        print(f"Failed to fetch rating history for {username}. Status code: {response.status_code}")

    return None

def days_to_cross_margin(username, target_lower, target_upper):
    rating_history = get_codeforces_rating_history(username)
    
    if rating_history is None:
        return None

    current_rating = None
    start_date = None
    end_date = None

    for entry in rating_history:
        rating = entry["newRating"]
        contest_time = entry["ratingUpdateTimeSeconds"]
        contest_date = datetime.fromtimestamp(contest_time)

        if rating <= target_lower:
            start_date = contest_date
        elif rating >= target_upper:
            end_date = contest_date
            break

    if start_date and end_date:
        days_taken = (end_date - start_date).days
        return days_taken
    else:
        print(f"{username} did not cross the rating margin within the specified range.")
        return None

def add_result_to_dataframe(df):
    df['Result'] = ""
    target_lower = 1200
    target_upper = 1400

    for index, row in df.iterrows():
        username = row['Handle']
        days_taken = days_to_cross_margin(username, target_lower, target_upper)
        
        if days_taken is not None:
            if 20 <= days_taken <= 40:
                df.at[index, 'Result'] = "Yes"
            else:
                df.at[index, 'Result'] = "No"
        else:
            df.at[index, 'Result'] = "N/A"

    return df

if __name__ == "__main__":
    
    df = add_result_to_dataframe(df)

    print(df)

sulvshuo did not cross the rating margin within the specified range.
JustInCase did not cross the rating margin within the specified range.
diordhd did not cross the rating margin within the specified range.
Failed to fetch rating history for you_xiao. Status code: 503
Failed to fetch rating history for callmepandey. Status code: 503
Failed to fetch rating history for Mbfibat. Status code: 503
Failed to fetch rating history for diobrando97. Status code: 503
dsakhdkas did not cross the rating margin within the specified range.
Failed to fetch rating history for yangbo01. Status code: 503
Bombylius did not cross the rating margin within the specified range.
SHEXINYI did not cross the rating margin within the specified range.
Failed to fetch rating history for bkd. Status code: 503
Failed to fetch rating history for horrlod. Status code: 503
Failed to fetch rating history for Astolfoo. Status code: 503
FranSkarica did not cross the rating margin within the specified range.
Failed to fetch

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [4]:
df.head()

,Handle,Max Rating,Result
0,wukaichen888,1922,No
1,Autreekt,1921,Yes
2,ButanlishiHuanwoAC,1981,No
3,h4d1,1921,No
4,kkpos0206,2104,No


In [5]:
df.to_csv('outdf5.csv', index=False)